In [1]:
import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=110,
    target_version=black.TargetVersion.PY310,
)

In [2]:
import os
import subprocess

import GooseSLURM as gs
import numpy as np

import lysis

In [3]:
experiment_type = np.dtype(
    [
        ("descriptor", np.str_, 40),
        ("file_code", np.str_, 40),
        ("forced_unbind", np.float_),
        ("average_bind_time", np.float_),
    ]
)
code_type = np.dtype([("file_code", np.str_, 40), ("descriptor", np.str_, 40), ("executable", np.str_, 40)])
run_type = np.dtype(
    [
        ("exp_code", np.str_, 15),
        ("experiment", np.str_, 40),
        ("code", np.str_, 40),
        ("seed", int),
        ("running_time", int),
    ]
)

In [4]:
experiments = np.array(
    [
        ("Physiological Kd", "", 8.52e-2, 27.8),
        ("10x smaller", "_Kd00020036", 0.5143, 277.8),
        ("10x bigger", "_Kd0236", 5.4e-3, 2.78),
    ],
    dtype=experiment_type,
)
programs = np.array(
    [
        # ("_along", "Diffuse along clot", "macro_Q2_diffuse_along"),
        # ("_always", "Always bind", "macro_Q2_always_rebind"),
        (
            "_into_and_along",
            "Diffuse into and along clot",
            "macro_Q2_diffuse_into_and_along",
        ),
        # ("_into", "Diffuse into clot", "macro_Q2_diffuse_into"),
    ],
    dtype=code_type,
)
runs = np.empty(15, dtype=run_type)

In [5]:
in_file_code = "_PLG2_tPA01{data_code}_Q2.dat"
out_file_code = "_PLG2_tPA01{data_code}{program_code}_Q2.dat"

In [6]:
runs = np.array(
    [
        # ("2023-02-02-2200", "Physiological Kd", "Diffuse along clot", 17109424, 0),
        # ("2023-02-02-2201", "Physiological Kd", "Always bind", 9965734, 0),
        ("2023-02-02-2203", "Physiological Kd", "Diffuse into and along clot", -2137354075, 0),
        # ("2023-02-02-2204", "Physiological Kd", "Diffuse into clot", -2135977853, 0),
        # ("2023-02-02-2205", "10x smaller", "Diffuse along clot", -848304637, 0),
        # ("2023-02-02-2206", "10x smaller", "Always bind", 1299539472, 0),
        ("2023-02-02-2208", "10x smaller", "Diffuse into and along clot", -854989241, 0),
        # ("2023-02-02-2209", "10x smaller", "Diffuse into clot", -850336215, 0),
        # ("2023-02-02-2210", "10x bigger", "Diffuse along clot", -1216563743, 0),
        # ("2023-02-02-2211", "10x bigger", "Always bind", 669985532, 0),
        ("2023-02-02-2213", "10x bigger", "Diffuse into and along clot - FIXED", -1212172957, 0),
        # ("2023-02-02-2214", "10x bigger", "Diffuse into clot", -1213352577, 0),
    ],
    dtype=run_type,
)

In [7]:
for run in runs:
    prog = programs[programs["descriptor"] == run["code"]]
    exper = experiments[experiments["descriptor"] == run["experiment"]]
    e = lysis.util.Experiment(
        os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "data"),
        experiment_code=run["exp_code"],
    )
    p = {
        "forced_unbind": float(exper["forced_unbind"]),
        "average_bind_time": float(exper["average_bind_time"]),
        "seed": int(run["seed"]),
        "total_time": int(run["running_time"]),
        "macro_version": "macro"
        + out_file_code[:-4].format(data_code=exper["file_code"][0], program_code=prog["file_code"][0]),
    }
    e.initialize_macro_param(p)
    e.to_file()

    with open(os.path.join(e.os_path, "README.md"), "w") as file:
        file.write(
            f""" ## {e.experiment_code}
Rerun of into and along "restricted move" experiments after 
    -  'restricted move' bug correction
    -  'passerby molecule' bug correction
    -  addition of extra output data
    -  changing the termination criteria from 'fixed time' to 'all fibers degraded' AND '95% of molecules have passed through'
    -  changing macro-unbind wait time to remaining leaving time.
    
The results here should be comparable to those in experiment 2023-02-02-22{e.experiment_code[-2:]}

This set of data is from the "{run['experiment']}" set.
"""
        )

    result = subprocess.run(
        ["cp", os.path.join("src", "fortran", prog["executable"][0] + ".f90"), e.os_path],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        capture_output=True,
    )
    print(result)

CompletedProcess(args=['cp', 'src/fortran/macro_Q2_always_rebind.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-02-02-2201'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_Q2_always_rebind.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-02-02-2206'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_Q2_always_rebind.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-02-02-2211'], returncode=0, stdout=b'', stderr=b'')


In [8]:
for run in runs:
    prog = programs[programs["descriptor"] == run["code"]]
    exper = experiments[experiments["descriptor"] == run["experiment"]]
    e = lysis.util.Experiment(
        os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "data"),
        experiment_code=run["exp_code"],
    )
    e.read_file()
    f = lysis.util.FortranMacro(
        exp=e,
        executable=os.path.join(
            "/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "bin", prog["executable"][0]
        ),
        in_file_code=in_file_code.format(data_code=exper["file_code"][0]),
        out_file_code=out_file_code.format(
            data_code=exper["file_code"][0], program_code=prog["file_code"][0]
        ),
    )
    comm = f.exec_command()
    comm += [
        ">",
        os.path.join(
            e.os_path,
            "macro"
            + out_file_code[:-4].format(data_code=exper["file_code"][0], program_code=prog["file_code"][0])
            + ".txt",
        ),
    ]
    comm = " ".join(comm)
    # job-options
    sbatch = {
        "job-name": "lysis-" + e.experiment_code,
        "out": os.path.join(e.os_path, "job.slurm-%j.out"),
        "nodes": 1,
        "mem": 3096,
        "ntasks": 1,
        "cpus-per-task": 1,
        "partition": "long",
    }
    script = ["module purge", "module load oneapi/compiler", comm]
    with open(os.path.join(e.os_path, "job.slurm"), "w") as file:
        file.write(gs.scripts.plain(script, **sbatch))
    result = subprocess.run(
        ["sbatch", os.path.join(e.os_path, "job.slurm")],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        capture_output=True,
    )
    print(result)

CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-02-02-2201/job.slurm'], returncode=0, stdout=b'Submitted batch job 2215087\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-02-02-2206/job.slurm'], returncode=0, stdout=b'Submitted batch job 2215088\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-02-02-2211/job.slurm'], returncode=0, stdout=b'Submitted batch job 2215089\n', stderr=b'')
